In [1]:
import numpy as np
import pinocchio as pin

# โหลดโมเดล
urdf_path = "/home/xero/Ambula/robot_ws/src/ambula_simulation/ambula_description/urdf/ambula_bot_v3/ambula.urdf"
model = pin.buildModelFromUrdf(urdf_path)
data = model.createData()

# ดึงค่าคงที่จากโมเดลครั้งเดียวตอน Initialize
x1, z1 = -0.015, 0.201  
z2 = 0.235              
L1 = np.sqrt(x1**2 + z1**2)
L2 = z2
beta = np.arctan2(-x1, z1) 

hip_id = model.joints[model.getJointId("left_hip_joint")].idx_q
knee_id = model.joints[model.getJointId("left_knee_joint")].idx_q

hip_lower = model.lowerPositionLimit[hip_id]
hip_upper = model.upperPositionLimit[hip_id]
knee_lower = model.lowerPositionLimit[knee_id]  
knee_upper = model.upperPositionLimit[knee_id]  

def get_length_from_knee(q_knee):
    C = np.cos(q_knee - beta)
    L_sq = L1**2 + L2**2 + 2 * L1 * L2 * C
    return np.sqrt(L_sq)

L_MAX_REACHABLE = get_length_from_knee(knee_lower) 
L_MIN_REACHABLE = get_length_from_knee(knee_upper) 

def compute_cartesian_ik(x_des, y_des, z_des):
    # 1. หาระยะกระจัดรวม (L_req)
    L_req = np.sqrt(x_des**2 + z_des**2)
    if L_req < 1e-5: L_req = 1e-5
        
    # 2. BOUND INPUT (Vector Scaling)
    L_safe = np.clip(L_req, L_MIN_REACHABLE, L_MAX_REACHABLE)
    x_safe = x_des * (L_safe / L_req)
    z_safe = z_des * (L_safe / L_req)
    
    # 3. หามุม Knee (กฎของโคไซน์)
    C_knee = (L_safe**2 - L1**2 - L2**2) / (2 * L1 * L2)
    C_knee = np.clip(C_knee, -1.0, 1.0)
    q_knee_math = beta + np.arccos(C_knee)
    q_knee = np.clip(q_knee_math, knee_lower, knee_upper)
    
    # 4. หามุม Hip (วิธีใหม่: Robust Trigonometry)
    # หาพิกัดล้อก่อนหมุนสะโพก (q_hip = 0)
    X_w = x1 - L2 * np.sin(q_knee)
    Z_w = z1 + L2 * np.cos(q_knee)
    
    # แก้สมการ Rotation Matrix ตรงๆ ด้วย Dot & Cross products
    # X_safe = X_w*cos(q_hip) - Z_w*sin(q_hip)
    # Z_safe = X_w*sin(q_hip) + Z_w*cos(q_hip)
    denominator = X_w**2 + Z_w**2
    cos_q_hip = (X_w * x_safe + Z_w * z_safe) / denominator
    sin_q_hip = (X_w * z_safe - Z_w * x_safe) / denominator
    
    q_hip_math = np.arctan2(sin_q_hip, cos_q_hip)
    q_hip = np.clip(q_hip_math, hip_lower, hip_upper)
    
    return q_hip, q_knee, x_safe, z_safe

# ==========================================
# ตรวจสอบความแม่นยำด้วย Forward Kinematics
# ==========================================
def test_cartesian_ik(x_target, z_target):
    q_hip, q_knee, x_safe, z_safe = compute_cartesian_ik(x_target, 0.0, z_target)
    
    q = pin.neutral(model)
    q[hip_id] = q_hip
    q[knee_id] = q_knee
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)
    
    base_id = model.getFrameId("base_link")
    hip_frame_id = model.getFrameId("left_hip_link")
    wheel_frame_id = model.getFrameId("left_wheel_link")
    
    R_base = data.oMf[base_id].rotation
    p_hip = data.oMf[hip_frame_id].translation
    p_wheel = data.oMf[wheel_frame_id].translation
    
    vec_local = R_base.T @ (p_wheel - p_hip)
    actual_x, actual_z = vec_local[0], vec_local[2]
    
    err_x = abs(x_safe - actual_x)
    err_z = abs(z_safe - actual_z)
    
    print(f"TARGET : x={x_target:+.3f}, z={z_target:+.3f}")
    print(f"SAFE   : x={x_safe:+.3f}, z={z_safe:+.3f} (IK Output)")
    print(f"ACTUAL : x={actual_x:+.3f}, z={actual_z:+.3f} (FK Result)")
    print(f"ERROR  : err_x={err_x:.6f}, err_z={err_z:.6f}")
    print(f"ANGLES : hip={q_hip:.3f}, knee={q_knee:.3f}\n")

print("--- Testing Robust Cartesian IK vs FK ---")
test_cartesian_ik(0.0, -0.30)
test_cartesian_ik(0.0, -0.1)
test_cartesian_ik(-0.05, -0.25)
test_cartesian_ik(0.40, -0.40) # สั่งยาวเกินจริง ทดสอบ Limit

--- Testing Robust Cartesian IK vs FK ---
TARGET : x=+0.000, z=-0.300
SAFE   : x=+0.000, z=-0.300 (IK Output)
ACTUAL : x=+0.000, z=-0.300 (FK Result)
ERROR  : err_x=0.000000, err_z=0.000000
ANGLES : hip=2.169, knee=1.707

TARGET : x=+0.000, z=-0.100
SAFE   : x=+0.000, z=-0.150 (IK Output)
ACTUAL : x=-0.000, z=-0.150 (FK Result)
ERROR  : err_x=0.000000, err_z=0.000000
ANGLES : hip=1.627, knee=2.531

TARGET : x=-0.050, z=-0.250
SAFE   : x=-0.050, z=-0.250 (IK Output)
ACTUAL : x=-0.050, z=-0.250 (FK Result)
ERROR  : err_x=0.000000, err_z=0.000000
ANGLES : hip=1.811, knee=1.977

TARGET : x=+0.400, z=-0.400
SAFE   : x=+0.227, z=-0.227 (IK Output)
ACTUAL : x=+0.227, z=-0.227 (FK Result)
ERROR  : err_x=0.000000, err_z=0.000000
ANGLES : hip=3.033, knee=1.571

